<a href="https://colab.research.google.com/github/Custardwh1pp/GoogleColab_DL/blob/main/lecture05_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第5回講義 宿題

## 課題

今Lessonで学んだことに工夫を加えて，CNNでより高精度なCIFAR10の分類器を実装してみましょう．

### 目標値

Accuracy 76%

### ルール


- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- Pytorchを利用して構いません．
- 今回から基本的にAPI制限はありません．
- ただしCNNベースでないモデル（Vision Transformerなど）やtorchvision等の既存モデル，学習済みモデルは用いないでください．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルをcsv形式で保存し，**Omnicampusの宿題タブから「第5回 畳み込みニューラルネットワーク」を選択して**提出してください．
    2. それに対応するpythonのコードを　ファイル＞ダウンロード＞.pyをダウンロード　から保存し，**Omnicampusの宿題タブから「第5回 畳み込みニューラルネットワーク (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコード全体をコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法

- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 作業ディレクトリを指定
work_dir = 'drive/MyDrive/Colab Notebooks/DLBasics2025_colab'

### データの読み込み（この部分は修正しないでください）

In [ ]:
import random

import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

#学習データ
x_train = np.load(work_dir + '/Lecture05/data/x_train.npy')
t_train = np.load(work_dir + '/Lecture05/data/t_train.npy')

#テストデータ
x_test = np.load(work_dir + '/Lecture05/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        data = x_train.astype('float32')
        self.x_train = []
        for i in range(data.shape[0]):
            self.x_train.append(Image.fromarray(np.uint8(data[i])))
        self.t_train = t_train
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.transform(self.x_train[idx]), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        data = x_test.astype('float32')
        self.x_test = []
        for i in range(data.shape[0]):
            self.x_test.append(Image.fromarray(np.uint8(data[i])))
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_test)

    def __getitem__(self, idx):
        return self.transform(self.x_test[idx])

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 畳み込みニューラルネットワーク(CNN)の実装

In [ ]:
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


fix_seed(seed=42)


class gcn():
    def __init__(self):
        pass

    def __call__(self, x):
        mean = torch.mean(x)
        std = torch.std(x)
        return (x - mean)/(std + 10**(-6))  # 0除算を防ぐ


val_size = 3000
train_data, val_data = torch.utils.data.random_split(trainval_data, [len(trainval_data)-val_size, val_size])  # 訓練データと検証データに分割


# 前処理を定義
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    gcn(),
])

transform = transforms.Compose([
    transforms.ToTensor(),
    gcn(),
])

# データセットに前処理を設定
trainval_data.transform = transform_train
test_data.transform = transform

batch_size = 64

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 256x4x4

            # Block 4 (New)
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 512x2x2 (changed from 4x4)
        )
        # Adjust input features for the classifier based on new output size
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2, 1024), # Adjusted for 2x2 output from last MaxPool
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

conv_net = SimpleCNN(num_classes=10)

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


conv_net.apply(init_weights)

n_epochs = 50 # エポック数を増加
lr = 0.01
device = 'cuda'

conv_net.to(device)
# SGDにモメンタムを追加
optimizer = optim.SGD(conv_net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
# 学習率スケジューラを追加
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
loss_function = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        optimizer.zero_grad()
        x = x.to(device)
        t = t.to(device)
        y = conv_net(x)
        loss = loss_function(y, t)
        loss.backward()
        optimizer.step()
        pred = y.argmax(dim=-1)

        acc_train += (pred == t).float().sum().item()
        n_train += t.size(0)
        losses_train.append(loss.tolist())

    # 学習率を更新
    scheduler.step()

    conv_net.eval()
    n_val = 0
    acc_val = 0
    with torch.no_grad(): # 検証中は勾配計算を無効化
        for x, t in dataloader_valid:
            x = x.to(device)
            t = t.to(device)
            y = conv_net(x)
            loss = loss_function(y, t)
            pred = y.argmax(dim=-1)

            acc_val += (pred == t).float().sum().item()
            n_val += t.size(0)
            losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


EPOCH: 0, Train [Loss: 1.721, Accuracy: 0.370], Valid [Loss: 1.314, Accuracy: 0.502]
EPOCH: 1, Train [Loss: 1.288, Accuracy: 0.544], Valid [Loss: 1.083, Accuracy: 0.625]
EPOCH: 2, Train [Loss: 1.043, Accuracy: 0.640], Valid [Loss: 0.898, Accuracy: 0.686]
EPOCH: 3, Train [Loss: 0.886, Accuracy: 0.700], Valid [Loss: 0.803, Accuracy: 0.715]
EPOCH: 4, Train [Loss: 0.774, Accuracy: 0.740], Valid [Loss: 0.765, Accuracy: 0.737]
EPOCH: 5, Train [Loss: 0.684, Accuracy: 0.770], Valid [Loss: 0.628, Accuracy: 0.787]
EPOCH: 6, Train [Loss: 0.620, Accuracy: 0.792], Valid [Loss: 0.733, Accuracy: 0.756]
EPOCH: 7, Train [Loss: 0.567, Accuracy: 0.808], Valid [Loss: 0.512, Accuracy: 0.829]
EPOCH: 8, Train [Loss: 0.524, Accuracy: 0.824], Valid [Loss: 0.570, Accuracy: 0.814]
EPOCH: 9, Train [Loss: 0.486, Accuracy: 0.836], Valid [Loss: 0.618, Accuracy: 0.797]
EPOCH: 10, Train [Loss: 0.451, Accuracy: 0.847], Valid [Loss: 0.543, Accuracy: 0.819]
EPOCH: 11, Train [Loss: 0.421, Accuracy: 0.859], Valid [Loss: 0.

In [ ]:
conv_net.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = conv_net.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv(work_dir + '/Lecture05/submission_pred_05.csv', header=True, index_label='id')

# Task
Implement and train a deeper CNN model (DeeperCNN) based on the existing `SimpleCNN` structure, then use this DeeperCNN model to predict on the test data and save the predictions to a CSV file.

## Define and Train Model 2 (DeeperCNN)

### Subtask:
Implement and train a deeper CNN model (DeeperCNN) based on the existing `SimpleCNN` structure.


**Reasoning**:
I will define the `DeeperCNN` class, instantiate it, apply weight initialization, and set up the optimizer, learning rate scheduler, and loss function. This covers the first six instructions of the subtask.



In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DeeperCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DeeperCNN, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 256x4x4

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # Output: 512x2x2

            # Block 5 (Added for DeeperCNN)
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) # Output: 512x1x1
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 1 * 1, 1024), # Adjusted for 1x1 output from last MaxPool
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

deeper_conv_net = DeeperCNN(num_classes=10)

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


deeper_conv_net.apply(init_weights)

n_epochs = 20 # エポック数を増加
lr = 0.01
device = 'cuda'

deeper_conv_net.to(device)
# SGDにモメンタムを追加
optimizer = optim.SGD(deeper_conv_net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
# 学習率スケジューラを追加
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
loss_function = nn.CrossEntropyLoss()


In [ ]:
import numpy as np

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    deeper_conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        optimizer.zero_grad()
        x = x.to(device)
        t = t.to(device)
        y = deeper_conv_net(x)
        loss = loss_function(y, t)
        loss.backward()
        optimizer.step()
        pred = y.argmax(dim=-1)

        acc_train += (pred == t).float().sum().item()
        n_train += t.size(0)
        losses_train.append(loss.tolist())

    # 学習率を更新
    scheduler.step()

    deeper_conv_net.eval()
    n_val = 0
    acc_val = 0
    with torch.no_grad(): # 検証中は勾配計算を無効化
        for x, t in dataloader_valid:
            x = x.to(device)
            t = t.to(device)
            y = deeper_conv_net(x)
            loss = loss_function(y, t)
            pred = y.argmax(dim=-1)

            acc_val += (pred == t).float().sum().item()
            n_val += t.size(0)
            losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

EPOCH: 0, Train [Loss: 1.643, Accuracy: 0.408], Valid [Loss: 1.338, Accuracy: 0.518]
EPOCH: 1, Train [Loss: 1.165, Accuracy: 0.590], Valid [Loss: 1.062, Accuracy: 0.635]
EPOCH: 2, Train [Loss: 0.923, Accuracy: 0.683], Valid [Loss: 0.811, Accuracy: 0.715]
EPOCH: 3, Train [Loss: 0.779, Accuracy: 0.735], Valid [Loss: 0.772, Accuracy: 0.737]
EPOCH: 4, Train [Loss: 0.680, Accuracy: 0.771], Valid [Loss: 0.675, Accuracy: 0.767]
EPOCH: 5, Train [Loss: 0.611, Accuracy: 0.794], Valid [Loss: 0.592, Accuracy: 0.798]
EPOCH: 6, Train [Loss: 0.550, Accuracy: 0.815], Valid [Loss: 0.564, Accuracy: 0.805]
EPOCH: 7, Train [Loss: 0.497, Accuracy: 0.832], Valid [Loss: 0.486, Accuracy: 0.825]
EPOCH: 8, Train [Loss: 0.452, Accuracy: 0.846], Valid [Loss: 0.537, Accuracy: 0.814]
EPOCH: 9, Train [Loss: 0.407, Accuracy: 0.865], Valid [Loss: 0.431, Accuracy: 0.851]
EPOCH: 10, Train [Loss: 0.374, Accuracy: 0.873], Valid [Loss: 0.418, Accuracy: 0.862]
EPOCH: 11, Train [Loss: 0.337, Accuracy: 0.884], Valid [Loss: 0.

In [ ]:
deeper_conv_net.eval()

t_pred_deepercnn = []
with torch.no_grad():
    for x in dataloader_test:
        x = x.to(device)
        y = deeper_conv_net.forward(x)
        pred = y.argmax(1).tolist()
        t_pred_deepercnn.extend(pred)

submission_deepercnn = pd.Series(t_pred_deepercnn, name='label')
submission_deepercnn.to_csv(work_dir + '/Lecture05/submission_pred_deepercnn.csv', header=True, index_label='id')

## Define and Train Model 3 (ResNet-like CNN)

### Subtask:
Residual Blockを取り入れたResNet風のCNNモデル（ResNet-like CNN）を定義し、訓練します。スキップコネクションを導入することで、深いネットワークでの勾配消失問題の緩和と精度向上を目指します。


**Reasoning**:
First, I need to define the `ResidualBlock` class, which is a fundamental building block for the ResNet-like CNN. This block will include convolutional layers, batch normalization, ReLU activation, and a skip connection to facilitate training of deeper networks.



In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity) # Skip connection
        out = self.relu(out)
        return out

print("ResidualBlock class defined.")

ResidualBlock class defined.


**Reasoning**:
Now that the `ResidualBlock` is defined, I will define the `ResNetLikeCNN` class by stacking these blocks. This class will include an initial convolutional layer, several residual blocks, and a final fully connected layer for classification, addressing instruction 2.



In [ ]:
class ResNetLikeCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNetLikeCNN, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(ResidualBlock, 512, 2, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

resnet_like_net = ResNetLikeCNN(num_classes=10)

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        if m.bias is not None: # Conv2d may not have bias if bias=False
            m.bias.data.fill_(0.0)


resnet_like_net.apply(init_weights)

n_epochs = 50 # エポック数を増加
lr = 0.01
device = 'cuda'

resnet_like_net.to(device)
# SGDにモメンタムを追加
optimizer_resnet = optim.SGD(resnet_like_net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
# 学習率スケジューラを追加
scheduler_resnet = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_resnet, T_max=n_epochs)
loss_function_resnet = nn.CrossEntropyLoss()

print("ResNetLikeCNN model defined, initialized, and setup for training.")

ResNetLikeCNN model defined, initialized, and setup for training.


**Reasoning**:
Now I will implement the training loop for the `ResNetLikeCNN` model as specified in instruction 9, iterating through the defined number of epochs, performing forward and backward passes, updating weights, and tracking loss and accuracy for both training and validation sets.



In [ ]:
import numpy as np

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    resnet_like_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        optimizer_resnet.zero_grad()
        x = x.to(device)
        t = t.to(device)
        y = resnet_like_net(x)
        loss = loss_function_resnet(y, t)
        loss.backward()
        optimizer_resnet.step()
        pred = y.argmax(dim=-1)

        acc_train += (pred == t).float().sum().item()
        n_train += t.size(0)
        losses_train.append(loss.tolist())

    # 学習率を更新
    scheduler_resnet.step()

    resnet_like_net.eval()
    n_val = 0
    acc_val = 0
    with torch.no_grad(): # 検証中は勾配計算を無効化
        for x, t in dataloader_valid:
            x = x.to(device)
            t = t.to(device)
            y = resnet_like_net(x)
            loss = loss_function_resnet(y, t)
            pred = y.argmax(dim=-1)

            acc_val += (pred == t).float().sum().item()
            n_val += t.size(0)
            losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

EPOCH: 0, Train [Loss: 1.481, Accuracy: 0.474], Valid [Loss: 1.121, Accuracy: 0.599]
EPOCH: 1, Train [Loss: 0.999, Accuracy: 0.649], Valid [Loss: 0.943, Accuracy: 0.674]
EPOCH: 2, Train [Loss: 0.792, Accuracy: 0.722], Valid [Loss: 0.840, Accuracy: 0.723]
EPOCH: 3, Train [Loss: 0.669, Accuracy: 0.766], Valid [Loss: 0.649, Accuracy: 0.782]
EPOCH: 4, Train [Loss: 0.583, Accuracy: 0.800], Valid [Loss: 0.648, Accuracy: 0.779]
EPOCH: 5, Train [Loss: 0.523, Accuracy: 0.819], Valid [Loss: 0.522, Accuracy: 0.828]
EPOCH: 6, Train [Loss: 0.469, Accuracy: 0.839], Valid [Loss: 0.510, Accuracy: 0.829]
EPOCH: 7, Train [Loss: 0.431, Accuracy: 0.850], Valid [Loss: 0.486, Accuracy: 0.830]
EPOCH: 8, Train [Loss: 0.404, Accuracy: 0.860], Valid [Loss: 0.447, Accuracy: 0.851]
EPOCH: 9, Train [Loss: 0.368, Accuracy: 0.873], Valid [Loss: 0.447, Accuracy: 0.848]
EPOCH: 10, Train [Loss: 0.346, Accuracy: 0.882], Valid [Loss: 0.457, Accuracy: 0.844]
EPOCH: 11, Train [Loss: 0.321, Accuracy: 0.888], Valid [Loss: 0.

KeyboardInterrupt: 

In [ ]:
import torch

print(f"CUDA is available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

In [ ]:
import pandas as pd

resnet_like_net.eval()

t_pred_resnet = []
with torch.no_grad():
    for x in dataloader_test:
        x = x.to(device)
        y = resnet_like_net.forward(x)
        pred = y.argmax(1).tolist()
        t_pred_resnet.extend(pred)

submission_resnet = pd.Series(t_pred_resnet, name='label')
submission_resnet.to_csv(work_dir + '/Lecture05/submission_pred_resnet.csv', header=True, index_label='id')